<a href="https://colab.research.google.com/github/alexeybondarev/MTS_Teta_Kaggle_Recommender/blob/main/MTS_Teta_Kaggle_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Открытие и предобработка

In [ ]:
import pandas as pd
from itertools import cycle, islice
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from more_itertools import pairwise
from tqdm.auto import tqdm
import scipy.sparse as sp

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
interactions = pd.read_csv("/content/gdrive/My Drive/interactions.csv")

In [ ]:
interactions.head()

,user_id,item_id,progress,rating,start_date
0,90133,82910,100,NaN,2018-01-01
1,159130,331068,70,NaN,2018-01-01
2,80061,26540,69,4.0,2018-01-01
3,12811,301895,16,NaN,2018-01-01
4,5778,127872,100,NaN,2018-01-01


In [ ]:
interactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1562617 entries, 0 to 1562616
Data columns (total 5 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   user_id     1562617 non-null  int64  
 1   item_id     1562617 non-null  int64  
 2   progress    1562617 non-null  int64  
 3   rating      323571 non-null   float64
 4   start_date  1562617 non-null  object 
dtypes: float64(1), int64(3), object(1)
memory usage: 59.6+ MB


In [ ]:
interactions.rating.value_counts()

5.0    212100
0.0     41787
4.0     39884
3.0     17965
1.0      6064
2.0      5771
Name: rating, dtype: int64

In [ ]:
interactions['start_date'] = pd.to_datetime(interactions['start_date'])

- what does zero rating mean? is it so bad or just not rated?
- is there any connection between progress and rating

In [ ]:
items = pd.read_csv('/content/gdrive/My Drive/items.csv')

items.head(2)

,id,title,genres,authors,year
0,248031,Ворон-челобитчик,"Зарубежные детские книги,Сказки,Зарубежная кла...",Михаил Салтыков-Щедрин,1886
1,256084,Скрипка Ротшильда,"Классическая проза,Литература 19 века,Русская ...",Антон Чехов,1894


In [ ]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63758 entries, 0 to 63757
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       63758 non-null  int64 
 1   title    63758 non-null  object
 2   genres   63753 non-null  object
 3   authors  56700 non-null  object
 4   year     49508 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.4+ MB


In [ ]:
items.year.value_counts()

2018          5382
2019          5053
2017          4738
2016          3734
2015          3554
              ... 
1962, 1974       1
1944, 1951       1
1947-1952        1
1960-1972        1
1860–1861        1
Name: year, Length: 1074, dtype: int64

- why are there items whithout author?

In [ ]:
users = pd.read_csv('/content/gdrive/My Drive/users.csv')
users.head()

,user_id,age,sex
0,0,45_54,1.0
1,1,25_34,0.0
2,2,45_54,0.0
3,3,65_inf,0.0
4,4,18_24,0.0


In [ ]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137254 entries, 0 to 137253
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   user_id  137254 non-null  int64  
 1   age      137244 non-null  object 
 2   sex      135640 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 3.1+ MB


In [ ]:
users.age.value_counts()

25_34     36701
35_44     31373
18_24     28413
45_54     22636
55_64     11803
65_inf     6318
Name: age, dtype: int64

In [ ]:
submission = pd.read_csv('/content/gdrive/My Drive/sample_submission.csv')
submission.head()

,Id,Predicted
0,10001,313 253 245 205 187 168 155 149 146 128 128 12...
1,10002,313 253 245 205 187 168 155 149 146 128 128 12...
2,100152,313 253 245 205 187 168 155 149 146 128 128 12...
3,100197,313 253 245 205 187 168 155 149 146 128 128 12...
4,100284,313 253 245 205 187 168 155 149 146 128 128 12...


In [ ]:
print(submission.shape)
print(submission['Id'].nunique())

(3074, 2)
3074


- To check if there is match between user and item id's accross the tables

In [ ]:
def get_names(indexes):
  name_list = {idx: df.title.loc[df.item_id == idx].iloc[0] for idx in indexes}
  return name_list

### Собираем все в одну таблицу

In [ ]:
df = interactions.merge(users, how='left', on='user_id')
df.head()

,user_id,item_id,progress,rating,start_date,age,sex
0,90133,82910,100,NaN,2018-01-01,55_64,0.0
1,159130,331068,70,NaN,2018-01-01,35_44,0.0
2,80061,26540,69,4.0,2018-01-01,25_34,0.0
3,12811,301895,16,NaN,2018-01-01,35_44,0.0
4,5778,127872,100,NaN,2018-01-01,45_54,1.0


In [ ]:
items.head(2)

,id,title,genres,authors,year
0,248031,Ворон-челобитчик,"Зарубежные детские книги,Сказки,Зарубежная кла...",Михаил Салтыков-Щедрин,1886
1,256084,Скрипка Ротшильда,"Классическая проза,Литература 19 века,Русская ...",Антон Чехов,1894


In [ ]:
items.rename(columns={'id': 'item_id'}, inplace=True)

In [ ]:
df = df.merge(items, how='left', on='item_id')

In [ ]:
df.head(5)

,user_id,item_id,progress,rating,start_date,age,sex,title,genres,authors,year
0,90133,82910,100,NaN,2018-01-01,55_64,0.0,Ведьма отмщения,Современные детективы,Галина Романова,2013
1,159130,331068,70,NaN,2018-01-01,35_44,0.0,Проклятый горн,"Героическое фэнтези,Боевое фэнтези",Алексей Пехов,2014
2,80061,26540,69,4.0,2018-01-01,25_34,0.0,Время – убийца,"Зарубежные детективы,Современные детективы",Мишель Бюсси,2016
3,12811,301895,16,NaN,2018-01-01,35_44,0.0,Великая Русь Средиземноморья. Книга III,Всемирная история,"Александр Саверский,Светлана Саверская",2014
4,5778,127872,100,NaN,2018-01-01,45_54,1.0,Об ограниченности западного «просвещения»,Книги по философии,Николай Федоров,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1562617 entries, 0 to 1562616
Data columns (total 11 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   user_id     1562617 non-null  int64         
 1   item_id     1562617 non-null  int64         
 2   progress    1562617 non-null  int64         
 3   rating      323571 non-null   float64       
 4   start_date  1562617 non-null  datetime64[ns]
 5   age         1333500 non-null  object        
 6   sex         1314763 non-null  float64       
 7   title       1562617 non-null  object        
 8   genres      1562559 non-null  object        
 9   authors     1458318 non-null  object        
 10  year        1314710 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(3), object(5)
memory usage: 143.1+ MB


In [ ]:
df.sort_values(by='user_id', inplace=True)

In [ ]:
df.head(2)

,user_id,item_id,progress,rating,start_date,age,sex,title,genres,authors,year
433018,0,315819,85,NaN,2018-07-24,45_54,1.0,Графология XXI века,"Практическая психология,Общая психология,Прогр...","Юрий Чернов,Илья Щеголев",2008
140396,0,89713,75,5.0,2018-03-08,45_54,1.0,Тридевятое царство. Удар Святогора,Боевое фэнтези,Денис Новожилов,2014


Получили df - таблицу со всеми возможными полями, с которой уже можно работать дальше, пропуски не заполнены.
Времены приведены к типу datatime

# Коллаборативная фильтрация с помощью скора и ALS

In [ ]:
df.head(2)

,user_id,item_id,progress,rating,start_date,age,sex,title,genres,authors,year
433018,0,315819,85,NaN,2018-07-24,45_54,1.0,Графология XXI века,"Практическая психология,Общая психология,Прогр...","Юрий Чернов,Илья Щеголев",2008
140396,0,89713,75,5.0,2018-03-08,45_54,1.0,Тридевятое царство. Удар Святогора,Боевое фэнтези,Денис Новожилов,2014


Для целей коллаборативной фильтрации можно придумать свой скор, который должен отражать время, популярностб книги и пр. Но это полная эвристика и не факт, что будет работать.
Попробую ввести следующий скор:
- недавность просмотра
- общая популярность
- наличие оценки (высокой)
- наличие факта progress

In [ ]:
min_date_30 = df['start_date'].max().normalize() - pd.DateOffset(days=30)
min_date_60 = df['start_date'].max().normalize() - pd.DateOffset(days=60)

In [ ]:
# Скор по недавности просмотра
conditions = [
    (df['start_date'] > min_date_30),
    (df['start_date'] > min_date_60),
    ]
values = [2, 1]

In [ ]:
df['score_1'] = np.select(conditions, values)

In [ ]:
df.head(2)

,user_id,item_id,progress,rating,start_date,age,sex,title,genres,authors,year,score_1
433018,0,315819,85,NaN,2018-07-24,45_54,1.0,Графология XXI века,"Практическая психология,Общая психология,Прогр...","Юрий Чернов,Илья Щеголев",2008,0
140396,0,89713,75,5.0,2018-03-08,45_54,1.0,Тридевятое царство. Удар Святогора,Боевое фэнтези,Денис Новожилов,2014,0


In [ ]:
df.score_1.value_counts()

0    1434872
1      64287
2      63458
Name: score_1, dtype: int64

In [ ]:
# Скор по общей популярности
#popular_list_10 = df.loc[df['start_date'].max().normalize() - pd.DateOffset(days=30), df['item_id']].value_counts().head(10).index.values
#popular_list_10 = df.loc[(df['start_date'] > df['start_date'].max().normalize() - pd.DateOffset(days=30)), 'item_id'].value_counts().head(10).index.values
popular_list_20 = df.loc[(df['start_date'] > df['start_date'].max().normalize() - pd.DateOffset(days=30)), 'item_id'].value_counts().head(20).index.values
#popular_list_100 = df.loc[(df['start_date'] > df['start_date'].max().normalize() - pd.DateOffset(days=30)), 'item_id'].value_counts().head(100).index.values
#popular_list_total_10 = df['item_id'].value_counts().head(10).index.values
popular_list_total_20 = df['item_id'].value_counts().head(20).index.values

In [ ]:
get_names(popular_list_20)

{33801: 'Два года каникул',
 50718: 'Москва и москвичи',
 55466: 'Понедельник начинается в субботу',
 79499: 'Женская война',
 80003: 'НИ СЫ. Восточная мудрость, которая гласит: будь уверен в своих силах и не позволяй сомнениям мешать тебе двигаться вперед',
 112869: 'Богатый папа, бедный папа. Роберт Кийосаки (обзор)',
 127019: 'Кому на Руси жить хорошо',
 143175: 'Хитрость',
 168963: 'Прислуга',
 184549: 'Записки юного врача',
 187325: 'Русские народные сказки',
 242176: 'Зулейха открывает глаза',
 267817: 'О любви',
 276903: 'Кавказский пленник',
 283713: 'Пикник на обочине',
 287060: 'Sapiens. Краткая история человечества',
 325439: 'Дни Турбиных',
 352049: 'История государства Российского. Том 2. От Великого князя Святополка до Великого князя Мстислава Изяславовича',
 357309: 'Вторая жизнь Уве',
 385281: 'Богатый папа, бедный папа'}

In [ ]:
# Скор по общей популярности
conditions = [
    df['item_id'].isin(popular_list_total_20),
    df['item_id'].isin(popular_list_20)
    ]
values = [1, 2]

In [ ]:
df['score_2'] = np.select(conditions, values)

In [ ]:
df.score_2.value_counts()

0    1514202
1      41983
2       6432
Name: score_2, dtype: int64

In [ ]:
# Скор по наличию оценки вообще и оценки 4 или больше
conditions = [
    df['rating'] >=4,
    ]
values = [1]

In [ ]:
df['score_3'] = np.select(conditions, values)

In [ ]:
df.score_3.value_counts()

0    1310633
1     251984
Name: score_3, dtype: int64

In [ ]:
# объединяем все скоры, добавив 1, если был хотя бы факт чтения
df['score'] = df['score_1'] + df['score_2'] + df['score_3'] + 1

In [ ]:
df.head(2)

,user_id,item_id,progress,rating,start_date,age,sex,title,genres,authors,year,score_1,score_2,score_3,score
433018,0,315819,85,NaN,2018-07-24,45_54,1.0,Графология XXI века,"Практическая психология,Общая психология,Прогр...","Юрий Чернов,Илья Щеголев",2008,0,0,0,1
140396,0,89713,75,5.0,2018-03-08,45_54,1.0,Тридевятое царство. Удар Святогора,Боевое фэнтези,Денис Новожилов,2014,0,0,1,2


In [ ]:
df.score.value_counts()

1    1161913
2     319404
3      68274
4      10097
5       2854
6         75
Name: score, dtype: int64

## Теперь можно провести ALS над этими скорами и посмотреть, что получится

In [ ]:
!pip install implicit

     |████████████████████████████████| 1.1MB 5.0MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp37-cp37m-linux_x86_64.whl size=3406438 sha256=18b5f71426723dde1e4db9374fc79795e79ca6344d6d613b7b95eeacf232bec3
  Stored in directory: /root/.cache/pip/wheels/bf/d4/ec/fd4f622fcbefb7521f149905295b2c26adecb23af38aa28217
Successfully built implicit


In [ ]:
df_user_item = df.groupby('user_id')['item_id'].agg(list).reset_index()

In [ ]:
df_user_item.head()

,user_id,item_id
0,0,"[315819, 89713, 389304, 153171, 48356, 75945, ..."
1,1,"[271192, 37820, 330390, 165038, 110325, 176447]"
2,2,"[344944, 111705, 319710, 198471, 361584, 182588]"
3,3,"[284107, 386271, 227278, 358784, 65005]"
4,4,"[184008, 119277, 289246]"


In [ ]:
users_inv_mapping = dict(enumerate(df['user_id'].unique()))
users_mapping = {v: k for k, v in users_inv_mapping.items()}
len(users_mapping)

158325

In [ ]:
items_inv_mapping = dict(enumerate(df['item_id'].unique()))
items_mapping = {v: k for k, v in items_inv_mapping.items()}
len(items_mapping)

63758

In [ ]:
def get_coo_matrix(df, 
                   user_col='user_id', 
                   item_col='item_id', 
                   weight_col=None, 
                   users_mapping=users_mapping, 
                   items_mapping=items_mapping):
    if weight_col is None:
        weights = np.ones(len(df), dtype=np.float32)
    else:
        weights = df[weight_col].astype(np.float32)

    interaction_matrix = sp.coo_matrix((
        weights, 
        (
            df[user_col].map(users_mapping.get), 
            df[item_col].map(items_mapping.get)
        )
    ))
    return interaction_matrix

In [ ]:
train_mat = get_coo_matrix(df, weight_col = 'score').tocsr()
train_mat

<158325x63758 sparse matrix of type '<class 'numpy.float32'>'
	with 1562505 stored elements in Compressed Sparse Row format>

Непонятно почему элементов в матрице только 1562505, хотя должно быть 1562617.
Разобрался, некоторые юзеры по 2 раза одну и ту же книгу помечали

In [ ]:
from implicit.als import AlternatingLeastSquares

In [ ]:
model = AlternatingLeastSquares(factors=20, regularization=0.1, iterations=50, use_gpu=False)

alpha = 20
data = (alpha * train_mat.T).astype('double')
model.fit(data)

In [ ]:
default_rec = [283713, 184549, 276903, 168963, 357309, 143175, 385281, 55466, 80003, 187325]

def generate_implicit_recs_mapper(model, train_matrix, N, user_mapping, item_inv_mapping):
    def _recs_mapper(user):
        try:
            user_id = user_mapping[user]
        except KeyError:
            return default_rec
        recs = model.recommend(user_id, 
                               train_matrix, 
                               N=N, 
                               filter_already_liked_items=True)
        return [int(item_inv_mapping[item]) for item, _ in recs]
    return _recs_mapper

In [ ]:
mapper = generate_implicit_recs_mapper(model, train_mat, 10, users_mapping, items_inv_mapping)

In [ ]:
recs = pd.DataFrame({
        'Id': submission['Id'].unique()
    })
recs['Predicted'] = recs['Id'].map(mapper)

In [ ]:
recs.head(5)

,Id,Predicted
0,10001,"[296208, 1194, 3484, 297168, 142970, 139365, 1..."
1,10002,"[377922, 80576, 263141, 112869, 61100, 299207,..."
2,100152,"[178529, 319710, 260913, 79815, 210450, 270299..."
3,100197,"[80576, 117325, 112869, 316483, 199827, 113621..."
4,100284,"[21769, 388868, 102108, 244187, 319710, 164263..."


In [ ]:
recs['Predicted'] = recs['Predicted'].map(lambda x: ' '.join(map(str, x)))

In [ ]:
recs.to_csv('/content/gdrive/My Drive/submission_ALS.csv', index=False)

In [ ]:
pred_pop.head()

,Id,Predicted
0,10001,283713 276903 184549 168963 357309 143175 3852...
1,10002,283713 276903 184549 168963 357309 143175 3852...
2,100152,283713 276903 184549 168963 357309 143175 3852...
3,100197,283713 276903 184549 168963 357309 143175 3852...
4,100284,283713 276903 184549 168963 357309 143175 3852...


In [ ]:
get_names(pred_pop.loc[0,'Predicted'])

{56877: 'Нахал',
 80003: 'НИ СЫ. Восточная мудрость, которая гласит: будь уверен в своих силах и не позволяй сомнениям мешать тебе двигаться вперед',
 168963: 'Прислуга',
 184549: 'Записки юного врача',
 276903: 'Кавказский пленник',
 283713: 'Пикник на обочине',
 287060: 'Sapiens. Краткая история человечества',
 334106: 'Подсознание может всё!',
 357309: 'Вторая жизнь Уве',
 385281: 'Богатый папа, бедный папа'}

In [ ]:
get_names(recs.loc[0][1])

{3484: 'Инферно',
 72612: 'S.N.U.F.F.',
 230500: 'Путешественник',
 249083: 'Шантарам',
 273112: 'Когда я вернусь, будь дома',
 296208: 'После – долго и счастливо',
 297168: 'До того как',
 308074: 'Евгений Онегин',
 316483: 'Мастер и Маргарита (Иллюстрированное издание)',
 376516: 'НИ СЫ. Будь уверен в своих силах и не позволяй сомнениям мешать тебе двигаться вперед'}

Почему-то методы коллаборативной фильтрации не работают, скор очень низкий. 0.00486

# Улучшенный most popular

In [ ]:
age_list = list(df.age.unique())
print(age_list)
sex_list = [0, 1]
print(sex_list)

['45_54', '25_34', '65_inf', '18_24', nan, '35_44', '55_64']
[0, 1]


In [ ]:
class PopularRecommender_Age():
    def __init__(self, max_K=100, days=30, item_column='item_id', dt_column='date', age_column='age', sex_column='sex'):
        self.max_K = max_K
        self.days = days
        self.item_column = item_column
        self.dt_column = dt_column
        self.age_column = age_column
        self.sex_column = sex_column
        self.recommendations = []
        self.recommendations_18 = []
        self.recommendations_25 = []
        self.recommendations_35 = []
        self.recommendations_45 = []
        self.recommendations_55 = []
        self.recommendations_65 = []
        
    def fit(self, df):
        min_date = df[self.dt_column].max().normalize() - pd.DateOffset(days=self.days)
        self.recommendations = df.loc[(df[self.dt_column] > min_date), self.item_column].value_counts().head(self.max_K).index.values
        self.recommendations_18 = df.loc[(df[self.dt_column] > min_date) & (df[self.age_column] == '18_24'), self.item_column].value_counts().head(self.max_K).index.values
        self.recommendations_25 = df.loc[(df[self.dt_column] > min_date) & (df[self.age_column] == '25_34'), self.item_column].value_counts().head(self.max_K).index.values
        self.recommendations_35 = df.loc[(df[self.dt_column] > min_date) & (df[self.age_column] == '35_44'), self.item_column].value_counts().head(self.max_K).index.values
        self.recommendations_45 = df.loc[(df[self.dt_column] > min_date) & (df[self.age_column] == '45_54'), self.item_column].value_counts().head(self.max_K).index.values
        self.recommendations_55 = df.loc[(df[self.dt_column] > min_date) & (df[self.age_column] == '55_64'), self.item_column].value_counts().head(self.max_K).index.values
        self.recommendations_65 = df.loc[(df[self.dt_column] > min_date) & (df[self.age_column] == '65_inf'), self.item_column].value_counts().head(self.max_K).index.values

    

In [ ]:
pop = PopularRecommender_Age(days=22, dt_column='start_date')
pop.fit(df)

In [ ]:
print(pop.recommendations_65[:10])
print(pop.recommendations[:10])
print(pop.recommendations_18[:10])
print(pop.recommendations_25[:10])

[283713 184549  55466 374648 276903  51423 143175 168963  50718 160349]
[283713 276903 184549 168963 357309 143175 385281  55466  80003 242176]
[ 80003 184549 283713 168963  56877 385281 390771 287060 310921 143175]
[357309  80003  56877 184549 276903 385281 168963 334106 262464 287060]


In [ ]:
get_names(pop.recommendations_18[:10])

{56877: 'Нахал',
 80003: 'НИ СЫ. Восточная мудрость, которая гласит: будь уверен в своих силах и не позволяй сомнениям мешать тебе двигаться вперед',
 143175: 'Хитрость',
 168963: 'Прислуга',
 184549: 'Записки юного врача',
 283713: 'Пикник на обочине',
 287060: 'Sapiens. Краткая история человечества',
 310921: 'Подсознание может всё!',
 385281: 'Богатый папа, бедный папа',
 390771: 'Шепот мертвых'}

In [ ]:
# список уже прочитанных книг для каждого пользователя
df_user_item = df.groupby('user_id')['item_id'].agg(list).reset_index()

In [ ]:
df_user_item.head()

,user_id,item_id
0,0,"[315819, 89713, 389304, 153171, 48356, 75945, ..."
1,1,"[271192, 37820, 330390, 165038, 110325, 176447]"
2,2,"[344944, 111705, 319710, 198471, 361584, 182588]"
3,3,"[284107, 386271, 227278, 358784, 65005]"
4,4,"[184008, 119277, 289246]"


In [ ]:
def diff(first, second):
        second = set(second)
        return [item for item in first if item not in second]

In [ ]:
class PopularRecommender_Age_Trend():
    def __init__(self, max_K=100, days=30, item_column='item_id', dt_column='date', age_column='age', sex_column='sex'):
        self.max_K = max_K
        self.days = days
        self.item_column = item_column
        self.dt_column = dt_column
        self.age_column = age_column
        self.sex_column = sex_column
        self.recommendations = []
        self.recommendations_18 = []
        self.recommendations_25 = []
        self.recommendations_35 = []
        self.recommendations_45 = []
        self.recommendations_55 = []
        self.recommendations_65 = []
        
    def fit(self, df):
        min_date = df[self.dt_column].max().normalize() - pd.DateOffset(days=self.days + self.days)
        max_date = df[self.dt_column].max().normalize() - pd.DateOffset(days=self.days)

        df_new = df.loc[(df[self.dt_column] > max_date)]
        df_old = df.loc[(df[self.dt_column] > min_date) & (df[self.dt_column] <= max_date)]
        recommendation = df.loc[(df[self.dt_column] > max_date), self.item_column].value_counts().head(10).index.values
        rec_list_old = {}
        rec_list_new = {}
        rec_sorted = {}
        for rec in recommendation:
            rec_list_old[rec] = df_old.loc[(df['item_id'] == rec), 'item_id'].count()
            rec_list_new[rec] = df_new.loc[(df['item_id'] == rec), 'item_id'].count()
            rec_sorted[rec] =  rec_list_new[rec] / rec_list_old[rec]
        rec_sorted = list(dict(sorted(rec_sorted.items(), key=lambda item: item[1], reverse=True)).keys())
        self.recommendations = rec_sorted
        # self.recommendations = df.loc[(df[self.dt_column] > min_date), self.item_column].value_counts().head(self.max_K).index.values
        
        df_new = df.loc[(df[self.dt_column] > max_date) & (df[self.age_column] == '18_24')]
        df_old = df.loc[(df[self.dt_column] > min_date) & (df[self.dt_column] <= max_date) & (df[self.age_column] == '18_24')]
        recommendation = df.loc[(df[self.dt_column] > max_date) & (df[self.age_column] == '18_24'), self.item_column].value_counts().head(10).index.values
        rec_list_old = {}
        rec_list_new = {}
        rec_sorted = {}
        for rec in recommendation:
            rec_list_old[rec] = df_old.loc[(df['item_id'] == rec), 'item_id'].count()
            rec_list_new[rec] = df_new.loc[(df['item_id'] == rec), 'item_id'].count()
            if rec_list_old[rec] != 0:
              rec_sorted[rec] =  rec_list_new[rec] / rec_list_old[rec]
            else:
              rec_sorted[rec] =  rec_list_new[rec] / 1
        rec_sorted = list(dict(sorted(rec_sorted.items(), key=lambda item: item[1], reverse=True)).keys())
        self.recommendations_18 = rec_sorted
        
        #self.recommendations_18 = df.loc[(df[self.dt_column] > min_date) & (df[self.age_column] == '18_24'), self.item_column].value_counts().head(self.max_K).index.values
        
        df_new = df.loc[(df[self.dt_column] > max_date) & (df[self.age_column] == '25_34')]
        df_old = df.loc[(df[self.dt_column] > min_date) & (df[self.dt_column] <= max_date) & (df[self.age_column] == '25_34')]
        recommendation = df.loc[(df[self.dt_column] > max_date) & (df[self.age_column] == '25_34'), self.item_column].value_counts().head(10).index.values
        rec_list_old = {}
        rec_list_new = {}
        rec_sorted = {}
        for rec in recommendation:
            rec_list_old[rec] = df_old.loc[(df['item_id'] == rec), 'item_id'].count()
            rec_list_new[rec] = df_new.loc[(df['item_id'] == rec), 'item_id'].count()
            if rec_list_old[rec] != 0:
              rec_sorted[rec] =  rec_list_new[rec] / rec_list_old[rec]
            else:
              rec_sorted[rec] =  rec_list_new[rec] / 1
        rec_sorted = list(dict(sorted(rec_sorted.items(), key=lambda item: item[1], reverse=True)).keys())
        self.recommendations_25 = rec_sorted

        #self.recommendations_25 = df.loc[(df[self.dt_column] > min_date) & (df[self.age_column] == '25_34'), self.item_column].value_counts().head(self.max_K).index.values
        
        df_new = df.loc[(df[self.dt_column] > max_date) & (df[self.age_column] == '35_44')]
        df_old = df.loc[(df[self.dt_column] > min_date) & (df[self.dt_column] <= max_date) & (df[self.age_column] == '35_44')]
        recommendation = df.loc[(df[self.dt_column] > max_date) & (df[self.age_column] == '35_44'), self.item_column].value_counts().head(10).index.values
        rec_list_old = {}
        rec_list_new = {}
        rec_sorted = {}
        for rec in recommendation:
            rec_list_old[rec] = df_old.loc[(df['item_id'] == rec), 'item_id'].count()
            rec_list_new[rec] = df_new.loc[(df['item_id'] == rec), 'item_id'].count()
            if rec_list_old[rec] != 0:
              rec_sorted[rec] =  rec_list_new[rec] / rec_list_old[rec]
            else:
              rec_sorted[rec] =  rec_list_new[rec] / 1
        rec_sorted = list(dict(sorted(rec_sorted.items(), key=lambda item: item[1], reverse=True)).keys())
        self.recommendations_35 = rec_sorted
        
        # self.recommendations_35 = df.loc[(df[self.dt_column] > min_date) & (df[self.age_column] == '35_44'), self.item_column].value_counts().head(self.max_K).index.values
        
        df_new = df.loc[(df[self.dt_column] > max_date) & (df[self.age_column] == '45_54')]
        df_old = df.loc[(df[self.dt_column] > min_date) & (df[self.dt_column] <= max_date) & (df[self.age_column] == '45_54')]
        recommendation = df.loc[(df[self.dt_column] > max_date) & (df[self.age_column] == '45_54'), self.item_column].value_counts().head(10).index.values
        rec_list_old = {}
        rec_list_new = {}
        rec_sorted = {}
        for rec in recommendation:
            rec_list_old[rec] = df_old.loc[(df['item_id'] == rec), 'item_id'].count()
            rec_list_new[rec] = df_new.loc[(df['item_id'] == rec), 'item_id'].count()
            if rec_list_old[rec] != 0:
              rec_sorted[rec] =  rec_list_new[rec] / rec_list_old[rec]
            else:
              rec_sorted[rec] =  rec_list_new[rec] / 1
        rec_sorted = list(dict(sorted(rec_sorted.items(), key=lambda item: item[1], reverse=True)).keys())
        self.recommendations_45 = rec_sorted
        
        #self.recommendations_45 = df.loc[(df[self.dt_column] > min_date) & (df[self.age_column] == '45_54'), self.item_column].value_counts().head(self.max_K).index.values
        
        df_new = df.loc[(df[self.dt_column] > max_date) & (df[self.age_column] == '55_64')]
        df_old = df.loc[(df[self.dt_column] > min_date) & (df[self.dt_column] <= max_date) & (df[self.age_column] == '55_64')]
        recommendation = df.loc[(df[self.dt_column] > max_date) & (df[self.age_column] == '55_64'), self.item_column].value_counts().head(10).index.values
        rec_list_old = {}
        rec_list_new = {}
        rec_sorted = {}
        for rec in recommendation:
            rec_list_old[rec] = df_old.loc[(df['item_id'] == rec), 'item_id'].count()
            rec_list_new[rec] = df_new.loc[(df['item_id'] == rec), 'item_id'].count()
            if rec_list_old[rec] != 0:
              rec_sorted[rec] =  rec_list_new[rec] / rec_list_old[rec]
            else:
              rec_sorted[rec] =  rec_list_new[rec] / 1
        rec_sorted = list(dict(sorted(rec_sorted.items(), key=lambda item: item[1], reverse=True)).keys())
        self.recommendations_55 = rec_sorted
        
        #self.recommendations_55 = df.loc[(df[self.dt_column] > min_date) & (df[self.age_column] == '55_64'), self.item_column].value_counts().head(self.max_K).index.values
        
        df_new = df.loc[(df[self.dt_column] > max_date) & (df[self.age_column] == '65_inf')]
        df_old = df.loc[(df[self.dt_column] > min_date) & (df[self.dt_column] <= max_date) & (df[self.age_column] == '65_inf')]
        recommendation = df.loc[(df[self.dt_column] > max_date) & (df[self.age_column] == '65_inf'), self.item_column].value_counts().head(10).index.values
        rec_list_old = {}
        rec_list_new = {}
        rec_sorted = {}
        for rec in recommendation:
            rec_list_old[rec] = df_old.loc[(df['item_id'] == rec), 'item_id'].count()
            rec_list_new[rec] = df_new.loc[(df['item_id'] == rec), 'item_id'].count()
            if rec_list_old[rec] != 0:
              rec_sorted[rec] =  rec_list_new[rec] / rec_list_old[rec]
            else:
              rec_sorted[rec] =  rec_list_new[rec] / 1
        rec_sorted = list(dict(sorted(rec_sorted.items(), key=lambda item: item[1], reverse=True)).keys())
        self.recommendations_65 = rec_sorted
        
        #self.recommendations_65 = df.loc[(df[self.dt_column] > min_date) & (df[self.age_column] == '65_inf'), self.item_column].value_counts().head(self.max_K).index.values
        

    def recommend(self, users=None, N=10):
        recs = self.recommendations_18[:N]
        if users is None:
            return recs
        else:
          return list(islice(cycle([recs]), len(users)))
    

In [ ]:
# Учтены возрастные группы и исключены книги, которые человек уже читал
def recommend_line(line, N=10):
        try:
          recs = pop.recommendations  
          if users[users.user_id == line.Id].age.item() == '18_24':  
            recs = pop.recommendations_18
          if users[users.user_id == line.Id].age.item() == '25_34':  
            recs = pop.recommendations_25
          if users[users.user_id == line.Id].age.item() == '35_44':  
            recs = pop.recommendations_35
          if users[users.user_id == line.Id].age.item() == '45_54':  
            recs = pop.recommendations_45
          if users[users.user_id == line.Id].age.item() == '55_64':  
            recs = pop.recommendations_55
          if users[users.user_id == line.Id].age.item() == '65_inf':  
            recs = pop.recommendations_65
          recs = diff(recs, df_user_item[df_user_item.user_id == line.Id].item_id.item())[:N]
        except:
          recs = pop.recommendations[:N]  
        return list(recs)

In [ ]:
pred_pop = pd.DataFrame({'Id': submission['Id'].unique()})
pred_pop['Predicted'] = pred_pop.apply(recommend_line, axis=1)


In [ ]:
pred_pop.head()

,Id,Predicted
0,10001,"[357309, 80003, 56877, 184549, 276903, 385281,..."
1,10002,"[283713, 276903, 184549, 168963, 357309, 14317..."
2,100152,"[357309, 80003, 56877, 184549, 276903, 385281,..."
3,100197,"[357309, 80003, 56877, 184549, 276903, 385281,..."
4,100284,"[283713, 184549, 276903, 143175, 55466, 168963..."


In [ ]:
pred_pop['Predicted'] = pred_pop['Predicted'].map(lambda x: ' '.join(map(str, x)))
pred_pop.head()

,Id,Predicted
0,10001,357309 80003 56877 385281 168963 262464 287060...
1,10002,168963 357309 385281 80003 242176 187325 28706...
2,100152,357309 80003 56877 385281 168963 262464 287060...
3,100197,357309 80003 56877 385281 168963 262464 287060...
4,100284,168963 357309 242176 112869 344047 91236 38528...


In [ ]:
pred_pop.to_csv('/content/gdrive/My Drive/submission_pop.csv', index=False)

- Эксперимент с 700 днями не удался, скор 0.00066
- При этом скор на 25 дней 0.01105
- Если честно, тупой перебор числа дней - это какая-то странная идея, тем не менее, поставил 22 дня и получил пока высший скор 0.01150
- По идее, можно кросс-валидацию запустить, разбить на валидационные 2 дня и прогнать через нее, надо только метрику мерить

# Implicit BM25




In [ ]:
from implicit.nearest_neighbours import CosineRecommender, BM25Recommender
import scipy.sparse as sp

In [ ]:
users_inv_mapping = dict(enumerate(interactions['user_id'].unique()))
users_mapping = {v: k for k, v in users_inv_mapping.items()}
len(users_mapping)

158325

In [ ]:
items_inv_mapping = dict(enumerate(interactions['item_id'].unique()))
items_mapping = {v: k for k, v in items_inv_mapping.items()}
len(items_mapping)

63758

In [ ]:
df.head(2)

,user_id,item_id,progress,rating,start_date,age,sex,title,genres,authors,year,score_1,score_2,score_3,score
433018,0,315819,85,NaN,2018-07-24,45_54,1.0,Графология XXI века,"Практическая психология,Общая психология,Прогр...","Юрий Чернов,Илья Щеголев",2008,0,0,0,1
140396,0,89713,75,5.0,2018-03-08,45_54,1.0,Тридевятое царство. Удар Святогора,Боевое фэнтези,Денис Новожилов,2014,0,0,1,2


В этом месте имеет смысл поиграться с весами, это максимум, что можно вытащить из базовой коллаборативной фильтрации. 
Еще можно комбинировать с most popular, но надо подумать как, например 3 most popular и остальное по коллаборативной с весами

In [ ]:
def define_weights(line):
  if line.rating >=2: return 2
  else:
    if line.progress >= 50: return 1.5
    else: return 1

In [ ]:
df['weights'] = df.apply(define_weights, axis=1)

In [ ]:
df.head(2)

,user_id,item_id,progress,rating,start_date,age,sex,title,genres,authors,year,score_1,score_2,score_3,score,weights
433018,0,315819,85,NaN,2018-07-24,45_54,1.0,Графология XXI века,"Практическая психология,Общая психология,Прогр...","Юрий Чернов,Илья Щеголев",2008,0,0,0,1,1.5
140396,0,89713,75,5.0,2018-03-08,45_54,1.0,Тридевятое царство. Удар Святогора,Боевое фэнтези,Денис Новожилов,2014,0,0,1,2,2.0


In [ ]:
def get_coo_matrix(df, 
                   user_col='user_id', 
                   item_col='item_id', 
                   weight_col=None, 
                   users_mapping=users_mapping, 
                   items_mapping=items_mapping):
    if weight_col is None:
        weights = np.ones(len(df), dtype=np.float32)
    else:
        weights = df[weight_col].astype(np.float32)

    interaction_matrix = sp.coo_matrix((
        weights, 
        (
            df[user_col].map(users_mapping.get), 
            df[item_col].map(items_mapping.get)
        )
    ))
    return interaction_matrix

In [ ]:
train_mat = get_coo_matrix(df, weight_col = 'score').tocsr()
train_mat

<158325x63758 sparse matrix of type '<class 'numpy.float32'>'
	with 1562505 stored elements in Compressed Sparse Row format>

In [ ]:
model = BM25Recommender(K=10)

In [ ]:
model.fit(train_mat.T)

In [ ]:
default_rec = [283713, 184549, 276903, 168963, 357309, 143175, 385281, 55466, 80003, 187325]

def generate_implicit_recs_mapper(model, train_matrix, N, user_mapping, item_inv_mapping):
    def _recs_mapper(user):
        try:
            user_id = user_mapping[user]
        except KeyError:
            return default_rec
        recs = model.recommend(user_id, 
                               train_matrix, 
                               N=N, 
                               filter_already_liked_items=True)
        return [int(item_inv_mapping[item]) for item, _ in recs]
    return _recs_mapper

In [ ]:
top_N = 10
mapper = generate_implicit_recs_mapper(model, train_mat, top_N, users_mapping, items_inv_mapping)
mapper


<function __main__.generate_implicit_recs_mapper.<locals>._recs_mapper>

In [ ]:
submission.head()
submission['Predicted'] = submission['Id'].map(mapper)

In [ ]:
submission.head()

,Id,Predicted
0,10001,"[296208, 297168, 235765, 237354, 132798, 26660..."
1,10002,"[378939, 327688, 43309, 54436, 240408, 85477, ..."
2,100152,"[167562, 113057, 137774, 202167, 44107, 342420..."
3,100197,"[179505, 202540, 39151, 72375, 335140, 73133, ..."
4,100284,"[168733, 343815, 273724, 170741, 240559, 31257..."


In [ ]:
submission['Predicted'] = submission['Predicted'].map(lambda x: ' '.join(map(str, x)))

In [ ]:
submission.head()

,Id,Predicted
0,10001,"[296208, 297168, 237354, 235765, 81107, 305706..."
1,10002,"[299207, 263141, 316483, 83150, 37074, 196820,..."
2,100152,"[105797, 313220, 377922, 296940, 81011, 389291..."
3,100197,"[391169, 263141, 206822, 377922, 213715, 17596..."
4,100284,"[102108, 388868, 164263, 170741, 312572, 36267..."


In [ ]:
submission.to_csv('/content/gdrive/My Drive/submission_if.csv', index=False)

## Отдельно коллаборативку для тех у кого очень много взамодействий. Скоры по ним другие возьму


In [ ]:
df.head(2)

,user_id,item_id,progress,rating,start_date,age,sex,title,genres,authors,year,score_1,score_2,score_3,score,weights
433018,0,315819,85,NaN,2018-07-24,45_54,1.0,Графология XXI века,"Практическая психология,Общая психология,Прогр...","Юрий Чернов,Илья Щеголев",2008,0,0,0,1,2.0
140396,0,89713,75,5.0,2018-03-08,45_54,1.0,Тридевятое царство. Удар Святогора,Боевое фэнтези,Денис Новожилов,2014,0,0,1,2,3.0


In [ ]:
def define_weights(line):
  if line.rating >=3: return 3
  else:
    if line.progress >= 50: 
      return 2
    elif line.item_id in popular_list_20:
      return 1.5
    elif line.start_date >= min_date_60:
      return 1.2
    else: return 1

In [ ]:
df['weights'] = df.apply(define_weights, axis=1)

In [ ]:
df['weights'].value_counts()

1.0    693848
2.0    543162
3.0    269949
1.2     50823
1.5      4835
Name: weights, dtype: int64

In [ ]:
train_mat_1 = get_coo_matrix(df, weight_col = 'weights').tocsr()
train_mat_1

<158325x63758 sparse matrix of type '<class 'numpy.float32'>'
	with 1562505 stored elements in Compressed Sparse Row format>

In [ ]:
model = BM25Recommender(K=10)

In [ ]:
model.fit(train_mat_1.T)

In [ ]:
top_N = 10
mapper = generate_implicit_recs_mapper(model, train_mat, top_N, users_mapping, items_inv_mapping)
mapper

<function __main__.generate_implicit_recs_mapper.<locals>._recs_mapper>

In [ ]:
top_20 = submission.copy()

In [ ]:
top_20['Predicted'] = top_20['Id'].map(mapper)

# собрать Most popular и collaborative

In [ ]:
pred_pop.head(2)

,Id,Predicted
0,10001,"[357309, 80003, 56877, 184549, 276903, 385281,..."
1,10002,"[283713, 276903, 184549, 168963, 357309, 14317..."


In [ ]:
submission.head(2)

,Id,Predicted
0,10001,"[296208, 297168, 235765, 237354, 132798, 26660..."
1,10002,"[378939, 327688, 43309, 54436, 240408, 85477, ..."


In [ ]:
recs = submission.copy()

In [ ]:
top_20.head(2)

,Id,Predicted
0,10001,"[296208, 297168, 235765, 237354, 79674, 132798..."
1,10002,"[378939, 43309, 327688, 54436, 234926, 12597, ..."


Надо найти пересечение между most popular и collaborative

In [ ]:
df_combined = recs.merge(pred_pop, on='Id',suffixes=(1, 2))

In [ ]:
df_combined.head(2)

,Id,Predicted1,Predicted2
0,10001,"[296208, 297168, 235765, 237354, 132798, 26660...","[357309, 80003, 56877, 184549, 276903, 385281,..."
1,10002,"[378939, 327688, 43309, 54436, 240408, 85477, ...","[283713, 276903, 184549, 168963, 357309, 14317..."


Вариант 1. Оставляем 10 most popular, но ранжируем, если они есть в персональных

In [ ]:
def combined_1(line):
  recs = []
  for item in line.Predicted2[:10]:
    if item in line.Predicted1:
      recs.append(item)
  for idx in range(10):
    if line.Predicted2[idx] not in recs:
      recs.append(line.Predicted2[idx])
      if len(recs) == 10:
        break
  return recs

In [ ]:
df_combined['Predicted'] = df_combined.apply(combined_1, axis=1)

вариант 2. Смешать мост популяр и коллаборативную

In [ ]:
def combined_2(line):
  recs = []
  for idx in range(5):
    recs.append(line.Predicted2[idx])
    recs.append(line.Predicted1[idx])
  return recs

In [ ]:
df_combined['Predicted'] = df_combined.apply(combined_2, axis=1)

вариант 3. Смешать мост полуляр и только те из коллаборативной, что есть в most polular

In [ ]:
def combined_3(line):
  recs = []
  idx_col = 0
  for idx in range(5):
    recs.append(line.Predicted2[idx])
    for i in range(idx_col, 20):
      if line.Predicted1[i] in line.Predicted2:
        recs.append(line.Predicted1[i])
        idx_col = i+1
        break
  lenght = len(recs)
  for i in range(10 - lenght):
    recs.append(line.Predicted2[i+ 10-lenght])
  return recs

In [ ]:
df_combined['Predicted'] = df_combined.apply(combined_3, axis=1)

Вариант 4. Most popular для тех у кого мало отзывов

In [ ]:
submission.head(2)

,Id,Predicted
0,10001,"[296208, 297168, 235765, 237354, 132798, 26660..."
1,10002,"[378939, 327688, 43309, 54436, 240408, 85477, ..."


In [ ]:
submission_list = list(submission.Id)

In [ ]:
df.head(2)

,user_id,item_id,progress,rating,start_date,age,sex,title,genres,authors,year,score_1,score_2,score_3,score
433018,0,315819,85,NaN,2018-07-24,45_54,1.0,Графология XXI века,"Практическая психология,Общая психология,Прогр...","Юрий Чернов,Илья Щеголев",2008,0,0,0,1
140396,0,89713,75,5.0,2018-03-08,45_54,1.0,Тридевятое царство. Удар Святогора,Боевое фэнтези,Денис Новожилов,2014,0,0,1,2


In [ ]:
df_int = df.groupby('user_id')['item_id'].count().reset_index()
df_int.head(2)

,user_id,item_id
0,0,10
1,1,6


In [ ]:
df_int[df_int.item_id >= 20].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 607 entries, 151 to 158127
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   user_id  607 non-null    int64
 1   item_id  607 non-null    int64
dtypes: int64(2)
memory usage: 14.2 KB


In [ ]:
df_int = df_int[df_int['user_id'].isin(submission_list)]

In [ ]:
df_int.head()

,user_id,item_id
24,24,10
40,41,5
66,67,5
99,101,5
105,107,5


In [ ]:
fig = px.histogram(
    df_int,
    x="item_id",
    width=600,
    height=300,
    title="Гистограмма по кол-ву рейтингов",
    labels={"x": "кол-во рейтингов у пользователя"},
)
fig.update_yaxes(title_text="количество")
fig.show()

In [ ]:
def combined_4(line):
  recs = []
  try:
    if df_int[df_int.user_id == line.Id]['item_id'].item() >= 12:
      recs = line.Predicted1
    else:
      recs = line.Predicted2
  except:
    print(line.Id)
    recs = line.Predicted2
  return recs

In [ ]:
df_combined['Predicted'] = df_combined.apply(combined_4, axis=1)

10309
156235


In [ ]:
df_combined.head(5)

,Id,Predicted1,Predicted2,Predicted
0,10001,"[296208, 297168, 235765, 237354, 132798, 26660...","[357309, 80003, 56877, 184549, 276903, 385281,...","[357309, 80003, 56877, 184549, 276903, 385281,..."
1,10002,"[378939, 327688, 43309, 54436, 240408, 85477, ...","[283713, 276903, 184549, 168963, 357309, 14317...","[283713, 276903, 184549, 168963, 357309, 14317..."
2,100152,"[167562, 113057, 137774, 202167, 44107, 342420...","[357309, 80003, 56877, 184549, 276903, 385281,...","[357309, 80003, 56877, 184549, 276903, 385281,..."
3,100197,"[179505, 202540, 39151, 72375, 335140, 73133, ...","[357309, 80003, 56877, 184549, 276903, 385281,...","[357309, 80003, 56877, 184549, 276903, 385281,..."
4,100284,"[168733, 343815, 273724, 170741, 240559, 31257...","[283713, 184549, 276903, 143175, 55466, 168963...","[283713, 184549, 276903, 143175, 55466, 168963..."


In [ ]:
df_combined.head(2)

,Id,Predicted1,Predicted2,Predicted3
0,10001,"[296208, 297168, 235765, 237354, 132798, 26660...","[357309, 80003, 56877, 184549, 276903, 385281,...","[296208, 297168, 235765, 237354, 79674, 132798..."
1,10002,"[378939, 327688, 43309, 54436, 240408, 85477, ...","[283713, 276903, 184549, 168963, 357309, 14317...","[378939, 43309, 327688, 54436, 234926, 12597, ..."


In [ ]:
df_combined = df_combined.merge(top_20, on='Id')

In [ ]:
df_combined.rename(columns={'Predicted': 'Predicted3'}, inplace=True)

In [ ]:
def combined_5(line):
  recs = []
  try:
    if df_int[df_int.user_id == line.Id]['item_id'].item() >= 20:
      recs = line.Predicted3
    elif df_int[df_int.user_id == line.Id]['item_id'].item() >= 12:
      recs = line.Predicted1
    else:
      recs = line.Predicted2
  except:
    print(line.Id)
    recs = line.Predicted2
  return recs

In [ ]:
df_combined['Predicted'] = df_combined.apply(combined_5, axis=1)

10309
156235


In [ ]:
df_submit = df_combined.drop(columns=['Predicted1', 'Predicted2', 'Predicted3'])

In [ ]:
df_submit.head()

,Id,Predicted
0,10001,"[357309, 80003, 56877, 184549, 276903, 385281,..."
1,10002,"[283713, 276903, 184549, 168963, 357309, 14317..."
2,100152,"[357309, 80003, 56877, 184549, 276903, 385281,..."
3,100197,"[357309, 80003, 56877, 184549, 276903, 385281,..."
4,100284,"[283713, 184549, 276903, 143175, 55466, 168963..."


In [ ]:
df_submit['Predicted'] = df_submit['Predicted'].map(lambda x: ' '.join(map(str, x)))

In [ ]:
df_submit.to_csv('/content/gdrive/My Drive/submission_pop_bm10_1.csv', index=False)

# EDA, how to apply collaborative filtering?

In [ ]:
interactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1562617 entries, 0 to 1562616
Data columns (total 5 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   user_id     1562617 non-null  int64         
 1   item_id     1562617 non-null  int64         
 2   progress    1562617 non-null  int64         
 3   rating      323571 non-null   float64       
 4   start_date  1562617 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 59.6 MB


In [ ]:
interactions.head(2)

,user_id,item_id,progress,rating,start_date
0,90133,82910,100,NaN,2018-01-01
1,159130,331068,70,NaN,2018-01-01


In [ ]:
rating = interactions.groupby("user_id").agg({'rating':'count', 'progress':'count'}).reset_index()

In [ ]:
users.user_id.count()

137254

In [ ]:
rating.rating.describe()

count    158325.000000
mean          2.043714
std           8.042959
min           0.000000
25%           0.000000
50%           1.000000
75%           2.000000
max         668.000000
Name: rating, dtype: float64

In [ ]:
fig = px.histogram(
    rating,
    x="rating",
    width=600,
    height=300,
    title="Гистограмма по кол-ву рейтингов",
    labels={"x": "кол-во рейтингов у пользователя"},
)
fig.update_yaxes(title_text="количество")
fig.show()

In [ ]:
fig = px.histogram(
    rating,
    x="progress",
    width=600,
    height=300,
    title="Гистограмма по кол-ву чтений",
    labels={"x": "кол-во рейтингов у пользователя"},
)
fig.update_yaxes(title_text="количество")
fig.show()

In [ ]:
# How many users with more then 10, 100 ratings?
print('total users with interations:', rating.user_id.count())
print('total users with non-zero ratings:', rating[rating.rating !=0].user_id.count())
print('total users with non-zero progress:', rating[rating.progress !=0].user_id.count())
print('total users with more then 10 ratings:', rating[rating.rating > 10].user_id.count())
print('total users with more then 100 ratings:', rating[rating.rating > 100].user_id.count())
print('total users with more then 20 progress:', rating[rating.progress > 20].user_id.count())
print('total users with more then 100 progress:', rating[rating.progress > 100].user_id.count())

total users with interations: 158325
total users with non-zero ratings: 113306
total users with non-zero progress: 158325
total users with more then 10 ratings: 2814
total users with more then 100 ratings: 194
total users with more then 20 progress: 7638
total users with more then 100 progress: 995


In [ ]:
submission.Id.count()

3074

The question is: the most prolific users with 100+ ratings or 100+ progress - are they helpful for collaborative filtering or harmful?

In [ ]:
# let's compare user_ids in all tables
print(users.duplicated(subset='user_id', keep='first').sum())
print(submission.duplicated(subset='Id', keep='first').sum())
print(list(set(submission.Id.unique()) - set(rating.user_id.unique())))
print(len(list(set(submission.Id.unique()) - set(rating[rating.rating !=0].user_id.unique()))))

#print(list(set(submission.Id.unique()) - set(users.user_id.unique())))

0
0
[156235, 10309]
744


- There are 2 users with cold start in submission counting all progress
- There are 744 users with cold start in submission counting ratings

In [ ]:
rating[rating.user_id == 156234]

,user_id,rating,progress
148986,156234,2,7
